In [1]:
# config
import os

# specify the shape of input for our network
image_shape = (28, 28, 1)

# specify the batch size and number of epochs
batch_size = 64
epochs = 100

# define path to base output directory
base_output = 'output'

# use base output path to derive the path to serialized model along with training history plot
model_path = os.path.sep.join([base_output, 'contrastlive_siamese_model'])
plot_path = os.path.sep.join([base_output, 'contrastlive_plot.png'])

In [2]:
# Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D

def build_siamese_model(input_shape, embedding_dim=48):
    # specify inputs for feature extractor network
    inputs = Input(input_shape)
    
    # define first set of CONV=>RELU=>POOL=>Dropout layers
    x = Conv2D(64, (2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)
    
    # define second set of CONV=>RELU=>POOL=>Dropout layers
    x = Conv2D(64, (2, 2), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    
    # prepare the final outputs
    pooled_output = GlobalAveragePooling2D()(x)
    outputs = Dense(embedding_dim)(pooled_output)

    # build the model
    model = Model(inputs, outputs)
    
    # return model to calling function
    return model

In [3]:
# Utils
#imports
from tensorflow.keras.datasets import mnist
import tensorflow.keras.backend as K
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

# image pair generation
def make_pairs(images, labels):
    # initialize 2 empty lists to hold (image, image) pairs and 
    # labels to indicate if a pair is positive or negative
    pair_images = []
    pair_labels = []
    
    # calculate the total number of classes present in dataset and then build a list of
    # indexes for each class label that provides indexes for all examples with a given label
    num_classes = len(np.unique(labels))
    idx = [np.where(labels==i)[0] for i in range(0, num_classes)]
    
    # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to current iteration
        current_image = images[idxA]
        label = labels[idxA]
        
        # randomly pic an image that belong to same class label
        idxB = np.random.choice(idx[label])
        pos_image = images[idxB]
        
        # prepare positive pair and update images and labels lists respectively
        pair_images.append([current_image, pos_image])
        pair_labels.append([1])
        
        # grab the indicies for each of the class labels not equal to current label and 
        # randomly pick an image corresponding to a label not equal to current label
        neg_idx = np.where(labels!=label)[0]
        neg_image = images[np.random.choice(neg_idx)]
        
        # prepare a negative pair of images and update our lists
        pair_images.append([current_image, neg_image])
        pair_labels.append([0])
        
    return (np.array(pair_images), np.array(pair_labels))

def euclidean_distance(vectors):
    # unpack vectors into seperate lists
    (featsA, featsB) = vectors
    
    # compute the sum of squared distances between vectors
    sum_squared = K.sum(K.square(featsA - featsB), axis = 1, keepdims=True)
    
    # return the euclidean distance between vectors
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

def plot_training(H, plot_path):
    # construct a plot that plots and saves training history
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(H.history['loss'], label='train_loss')
    plt.plot(H.history['val_loss'], label='val_loss')
    plt.plot(H.history['accuracy'], label='train_acc')
    plt.plot(H.history['val_accuracy'], label='val_accuracy')
    plt.title('Training Loss and Accuracy')
    plt.xlabel('Epoch #')
    plt.ylabel('Loss/Accuracy')
    plt.legend(loc='lower left')
    plt.savefig(plot_path)

In [ ]:
# contrastlive_loss
import tensorflow.keras.backend as K
import tensorflow as tf

def constrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted class label
    # type (otherwise we run the riskof having two seperate data types, causing 
    # tensorflow to error out)
    y = tf.cast(y, preds.dtype)
    
    # calculate the constrastive loss between true labels and predicted labels
    squared_preds = K.square(preds)
    squared_margin = K.square(K.maximum(margin-preds, 0))
    loss = K.mean(y * squared_preds + (1 - y) * squared_margin)
    
    return loss